# Avito demand prediction challenge

Avito is one of Russia’s largest classified advertisements website.
The idea of this challenge is to predict the demand for a product.

<h3>Details:</h3>
<p>In their fourth Kaggle competition, Avito is challenging you to predict demand for an online advertisement based on its full description (title, description, images, etc.), its context (geographically where it was posted, similar ads already posted) and historical demand for similar ads in similar contexts. With this information, Avito can inform sellers on how to best optimize their listing and provide some indication of how much interest they should realistically expect to receive.
</p>
<h3>Link:</h3>
<p>Full description of the challenge is available here:</p>
<a href="https://www.kaggle.com/c/avito-demand-prediction/overview" target="_blank">https://www.kaggle.com/c/avito-demand-prediction/overview</a>


## Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import time
# Models Packages
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Gradient Boosting
import lightgbm as lgb
# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords
# Viz
import seaborn as sns
import matplotlib.pyplot as plt
import string
import sys

In [2]:
try:
    ## Occasionally (dev purpose only)
    sys.path.insert(0, "../..")
    import aisimplekit
except ModuleNotFoundError as err:
    print("""[err] {err}""".format(err=err))
    print("""Try: `pip install aisimplekit`""")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from aisimplekit.features.stats import *
from aisimplekit.utils.memory import reduce_mem_usage
from aisimplekit.cv.cv_kfold import cross_validate_lgbm_regressor
import aisimplekit.features.tfidf as tfidf
from aisimplekit.dnn.rnn import RnnTextModel, RnnModelType

Using TensorFlow backend.


## Helper: Loading data files

In [5]:
def load_data(gp, downsample_ratio=None, folder='../input/avito-demand-prediction'):
    """ """
    print('Loading train/test')
    train = pd.read_csv(folder+'/train.csv', index_col = "item_id", parse_dates = ["activation_date"])
    test = pd.read_csv(folder+'/test.csv', index_col = "item_id", parse_dates = ["activation_date"])
    train_index = train.index
    test_index = test.index

    if downsample_ratio is not None:
        print('Downsampling: %s' % downsample_ratio)
        assert downsample_ratio > 1.0
        from sklearn.utils import resample
        train = resample(train, replace=False, n_samples=int(len(train)/downsample_ratio), random_state=123)
        test = resample(test, replace=False, n_samples=int(len(test)/downsample_ratio), random_state=123)
        train_index = train.index
        test_index = test.index

    if gp is not None:
        print('Merging train/gp and test/gp')
        train = train.reset_index().merge(gp, on='user_id', how='left').set_index('item_id')
        test = test.reset_index().merge(gp, on='user_id', how='left').set_index('item_id')

    return (train, test, train_index, test_index)

## Helpers: Feature Extraction: User aggregated metrics

In [6]:
def compute_aggregated_metrics(folder='../input/avito-demand-prediction', save_features=False):
    """ Computes 3 features:
    - `avg_times_up_user` - how often the average item of the user has been put up for sale.
    - `avg_days_up_user` - the average number of days an item from the user has been put up for sale.
    - `n_user_items` - the number of items the user has put up for sale.
    """
    print('1/6 Loading data')
    used_cols = ['item_id', 'user_id', 'price']
    train = pd.read_csv(folder+'/train.csv', usecols=used_cols)
    train_active = pd.read_csv(folder+'/train_active.csv', usecols=used_cols)
    test = pd.read_csv(folder+'/test.csv', usecols=used_cols)
    test_active = pd.read_csv(folder+'/test_active.csv', usecols=used_cols)

    print('2.1/6 Building concatenated dataframe: all_samples')
    all_samples = pd.concat([train,train_active,test,test_active]).reset_index(drop=True)
    all_samples.drop_duplicates(['item_id'], inplace=True)
    del train_active; del test_active; gc.collect()

    print('2.2/Aggregating price by user')
    gp2 = None
    gp3 = None
    if False:
        gp2 = all_samples.groupby('user_id')['price'].mean()
        gp3 = all_samples.groupby('user_id')['price'].max()
    all_samples.drop(['price'], inplace=True, axis=1)
    
    ## concatenate the train and test period data to one dataframe for easier processing
    print('2.3/6 Loading/Building concatenated dataframe: all_periods')
    train_periods = pd.read_csv(folder+'/periods_train.csv', parse_dates=['date_from', 'date_to'])
    test_periods = pd.read_csv(folder+'/periods_test.csv', parse_dates=['date_from', 'date_to'])
    all_periods = pd.concat([train_periods,test_periods])
    del train_periods; del test_periods; gc.collect()

    ## Compute features
    print('3/6 Computing: days_up, days_up_sum, times_put_up')
    all_periods['days_up'] = all_periods['date_to'].dt.dayofyear - all_periods['date_from'].dt.dayofyear

    gp = all_periods.groupby(['item_id'])[['days_up']]
    gp_df = pd.DataFrame()
    gp_df['days_up_sum'] = gp.sum()['days_up']
    gp_df['times_put_up'] = gp.count()['days_up']
    gp_df.reset_index(inplace=True)
    gp_df.rename(index=str, columns={'index': 'item_id'})

    print('4/6 Merging')
    all_periods.drop_duplicates(['item_id'], inplace=True)
    all_periods = all_periods.merge(gp_df, on='item_id', how='left')
    del gp; del gp_df; gc.collect()

    ## We have an interesting but kind of useless feature now. As seen in the second venn diagram, there is no overlap at all between `train_active` (and with that `train_periods`) and `train` concerning *item* IDs.
    ## For the feature to become useful, we somehow have to associate an item ID with a user ID.
    all_periods = all_periods.merge(all_samples, on='item_id', how='left')

    print('5/6 Computing metric 1 and 2')
    ## Group items: Metrics 1/3 and 2/3
    gp = all_periods.groupby(['user_id'])[['days_up_sum', 'times_put_up']].mean().reset_index() \
        .rename(index=str, columns={'days_up_sum': 'avg_days_up_user', 'times_put_up': 'avg_times_up_user'})
    ## Metric 3/3
    ## For our last feature, `n_user_items`, we just group by user ID and count the number of items.
    ## We have to be careful to use `all_samples` instead of `all_periods` here because the latter does not contain the `train.csv` and `test.csv` samples.
    print('6/6 Computing metric 3')
    n_user_items = all_samples.groupby(['user_id'])[['item_id']].count().reset_index() \
        .rename(index=str, columns={'item_id': 'n_user_items'})
    gp = gp.merge(n_user_items, on='user_id', how='outer')

    ## Save the features
    if save_features is True:
        gp.to_csv('aggregated_features.csv', index=False)
    ## Cleanup
    del all_samples; del all_periods; del train; del test; gc.collect()
    return (gp, gp2, gp3)

## Helpers: Feature Extraction:
### User statistic features
### Category statistic features
### Statistics about geography counts, image counts.

In [7]:
def add_user_features(df):
    ## Q2 - How many posts per user, divided by posts from his (region or city) ?
    df = do_count(df, ['region', 'city'], 'X3', show_max=True);
    df = do_count(df, ['user_id'], 'X4', show_max=True)
    df['X5'] = df['X4']/df['X3']; df.drop(['X3','X4'],axis=1,inplace=True) # X5: 
    """ Q3 - How many posts having a description (or image or title or ...), divided by user's total posts ? """
    df = do_count(df, ['user_id'], 'X6', show_max=True)
    df = do_countuniq(df, ['user_id'], 'image_top_1', 'X7', show_max=True)
    df['X8'] = df['X7']/df['X6']; df.drop(['X7','X6'],axis=1,inplace=True) #

    cols = ['description_num_chars', 'description_num_words', 'description_num_unique_words', 'description_words_vs_unique',
            'title_num_chars', 'title_num_words', 'title_num_unique_words', 'title_words_vs_unique']
    for col in cols:
        df = do_mean(df, ['user_id'], col, 'mean_user_%s'%col, show_max=True)
        df = do_mean(df, ['category_name'], col, 'mean_category_%s'%col, show_max=True)
        df['ratio_mean_user-cat_%s'%col] = df['mean_user_%s'%col]/df['mean_category_%s'%col]
        df.drop(['mean_user_%s'%col, 'mean_category_%s'%col],axis=1,inplace=True)
    return df   ## Unique counts, Means; Variances, Min/max/Median ; Top-ranked (categorical); First/Last .. ; Previous/Next

def add_category_features(df):
    df['price_rank'] = df.groupby(['category_name'])['price'].rank(ascending=True) # 0.229789
    return df

def add_other_features(df):
    if True:
        df = do_count(df, ['region'], 'T0', show_max=True)
        df = do_count(df, ['city'], 'T1', show_max=True)
        df['T2'] = df['T1']/df['T0']; df.drop(['T0','T1'],axis=1,inplace=True)
    if True:
        df = do_count(df, ['image_top_1'], 'T5', show_max=True) # 0.229818
        df = do_count(df, ['image_top_1', 'category_name'], 'T8', show_max=True) # 0.229776
    return df

def add_stat_features(df):
    df = add_user_features(df)
    df = add_category_features(df)
    df = add_other_features(df)
    return df

## Common settings

In [8]:
gp = None
DEV = True
VALID = False # helps to find the best num_rounds !
n_rounds = 2401 # identified during validation, WARNING: set early_stopping_rounds to 50 !
#data_dir = '../input/avito-demand-prediction'
data_dir = '~/.kaggle/competitions/avito-demand-prediction/'

USE_IMAGE_FEATURES = False 

## Dev Settings

In [9]:
optimize_memory = False
max_features = 8000 # TF-IDF: Counts

## Dev settings
early_stopping_rounds = 15 # 50
learning_rate = 0.05 # 0.019
num_leaves = 128
downsample_ratio = 8.0

with_user_agg = False
with_tfidf = True

main_model = 'dnn' # 'lgb' or 'dnn'
analyzer = 'word' # 'word' or 'char'

## Submit Settings

In [10]:
if DEV is False:
    ## Same settings for VALID and SUBMIT
    early_stopping_rounds = 50
    learning_rate = 0.019
    num_leaves = 250
    downsample_ratio = None

    with_user_agg = True

    if main_model == 'dnn':
        with_tfidf = False
        
    optimize_memory = True
    max_features = None
    sub_filename = "lgb-base-2.csv"

## Loading (or computing): user aggregated features

In [11]:
%%time
gp = None
gp2 = None
gp3 = None
agg_cols = []

if gp is not None:
    print('Reusing gp.')
elif with_user_agg is True:
    (gp, gp2, gp3) = compute_aggregated_metrics(folder=data_dir, save_features=False)
    agg_cols = list(gp.columns)[1:]
else:
    print('Not using aggregated metrics.')

Not using aggregated metrics.
CPU times: user 544 µs, sys: 58 µs, total: 602 µs
Wall time: 481 µs


## Load Train/Test Data

In [12]:
%%time
(training, testing, traindex, testdex) = load_data(gp, downsample_ratio=downsample_ratio, folder=data_dir)

if gp is not None and optimize_memory is True:
    del(gp)
    gc.collect()

y = training.deal_probability.copy().clip(0.0, 1.0)
training.drop("deal_probability",axis=1, inplace=True)
print('Train shape: {} Rows, {} Columns'.format(*training.shape))
print('Test shape: {} Rows, {} Columns'.format(*testing.shape))
print("Combining Train and Test")

df = pd.concat([training,testing],axis=0)
del training, testing
gc.collect()
print('Final dataframe shape: {} Rows, {} Columns'.format(*df.shape))

Loading train/test
Downsampling: 8.0
Train shape: 187928 Rows, 16 Columns
Test shape: 63554 Rows, 16 Columns
Combining Train and Test
Final dataframe shape: 251482 Rows, 16 Columns
CPU times: user 47.3 s, sys: 3.54 s, total: 50.9 s
Wall time: 54 s


## Load pre-computed image features

In [13]:
# FIXME !
if USE_IMAGE_FEATURES is True:
    df_img = pd.read_csv('../input/trainimgfeatv2/train_imgfeat_v2.csv')
    df_img = df_img.rename(columns={'Unnamed: 0.1': 'image'})
    df_img = df_img.drop('Unnamed: 0', axis=1)
    df_img['image'] = df_img['image'].apply(lambda x: x.rstrip('.jpg'))
    df_img['image'] = df_img['image'].apply(str)
    df = df.reset_index().merge(df_img, on='image', how='left').set_index('item_id')
    for col in df_img.columns:
        df[col].fillna(-1.0, inplace=True)
    del(df_img)
    gc.collect()

## [1/4] Feature Engineering: Simple transformer features (np.log, dates) + encoding.

In [14]:
%%time
print("Feature Engineering")
df["price"] = np.log(df["price"]+0.001)
df["price"].fillna(-999,inplace=True)
df["image_top_1"].fillna(-999,inplace=True)

print("Creating Time Variables..")
df["Weekday"] = df['activation_date'].dt.weekday
df["Weekd of Year"] = df['activation_date'].dt.week
df["Day of Month"] = df['activation_date'].dt.day

# Create Validation Index and Remove Dead Variables
# training_index = df.loc[df.activation_date<=pd.to_datetime('2017-04-07')].index
# validation_index = df.loc[df.activation_date>=pd.to_datetime('2017-04-08')].index
df.drop(["activation_date", "image"],axis=1,inplace=True)

print("Encoding categorical variables")
categorical = ["user_id", "region", "city", "parent_category_name", "category_name",
               "user_type", "image_top_1", "param_1", "param_2", "param_3"]
lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col] = lbl.fit_transform(df[col].astype(str))

Feature Engineering
Creating Time Variables..
Encoding categorical variables
CPU times: user 4.35 s, sys: 72.3 ms, total: 4.42 s
Wall time: 4.57 s


## [2/4] Feature Engineering: Simple Text Features

In [15]:
%%time
# Meta Text Features
textfeats = ["description", "title"]
count = lambda l1,l2: sum([1 for x in l1 if x in l2])

for cols in textfeats:
    df[cols] = df[cols].astype(str) 
    df[cols] = df[cols].astype(str).fillna('missing') # FILL NA
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words
    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_desc_punct'] = df[cols].apply(lambda x: count(x, set(string.punctuation)))

for col in agg_cols:
    df[col].fillna(-1, inplace=True)

CPU times: user 19.8 s, sys: 65.6 ms, total: 19.9 s
Wall time: 20.9 s


## [3/4] Feature Engineering: Stats features

In [16]:
%%time
## Add statistical features
df = add_stat_features(df)

Aggregating by  ['region', 'city'] ...
X3 max value =  10743
Aggregating by  ['user_id'] ...
X4 max value =  188
Aggregating by  ['user_id'] ...
X6 max value =  188
Counting unique  image_top_1  by  ['user_id'] ...
X7 max value =  91
Calculating mean of  description_num_chars  by  ['user_id'] ...
mean_user_description_num_chars max value =  3144.0
Calculating mean of  description_num_chars  by  ['category_name'] ...
mean_category_description_num_chars max value =  810.3571428571429
Calculating mean of  description_num_words  by  ['user_id'] ...
mean_user_description_num_words max value =  597.0
Calculating mean of  description_num_words  by  ['category_name'] ...
mean_category_description_num_words max value =  113.14880952380952
Calculating mean of  description_num_unique_words  by  ['user_id'] ...
mean_user_description_num_unique_words max value =  333.0
Calculating mean of  description_num_unique_words  by  ['category_name'] ...
mean_category_description_num_unique_words max value =

## [4/4] Feature Engineering: TF-IDF Text Features

In [17]:
transformer_spec = {
    'description': {
        'vectorizer': tfidf.TRANSFORMER_TFIDF,
        'ngram_range': (1,2),
        'max_features': 17000,
        'kwargs': {} # overridable
    },
    'title': {
        'vectorizer': tfidf.TRANSFORMER_COUNT,
        'ngram_range': (1,2),
        'max_features': max_features,
        'kwargs': None # no additional named args
    }
}

In [18]:
%%time
(vectorizer, df_tfidf, tf_vocab) = tfidf.compute_features(df, transformer_spec, analyzer='word', stop="russian")
print('#vocab_size: %d' % len(tf_vocab))

#vocab_size: 25000
CPU times: user 2min 27s, sys: 2.29 s, total: 2min 30s
Wall time: 2min 42s


## Train Model: LGB

In [19]:
%%time
if main_model == 'lgb':
    # Drop Text Cols
    try:
        df.drop(textfeats, axis=1, inplace=True)
    except KeyError as e:
        print(e)

    # Reduce Memory (See function up top)
    df = reduce_mem_usage(df)

    # Combine Dense Features with Sparse Text Bag of Words Features
    print('Concatenating base features + tfidf features..')
    if df_tfidf is not None:
        X = hstack([csr_matrix(df.loc[traindex,:].values), df_tfidf[0:traindex.shape[0]]]) # Sparse Matrix
        testing = hstack([csr_matrix(df.loc[testdex,:].values), df_tfidf[traindex.shape[0]:]])
        predictors = df.columns.tolist() + tf_vocab
    else:
        X = hstack([csr_matrix(df.loc[traindex,:].values)]) # Sparse Matrix
        testing = hstack([csr_matrix(df.loc[testdex,:].values)])
        predictors = df.columns.tolist()

    del(df)
    del(df_tfidf)
    gc.collect()

    lgbm_params =  {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse', # 'max_depth': 15,
        'num_leaves': num_leaves,
        'feature_fraction': 0.50,
        'bagging_fraction': 0.70, # 'bagging_freq': 5,
        'learning_rate': learning_rate,
        'verbose': 0
    }

    # Training and Validation Set
    modelstart = time.time()
    print('Training..')
    if VALID == True:
        print('Mode: Dev with Cross-validation')
        ## CV: identify best tuning params, and features
        (scores, best_num_rounds) = cross_validate_lgbm_regressor(
                                        X.tocsr(), y, folds=3, repeats=1, predictors=predictors,
                                        categorical=categorical, lgbm_params=lgbm_params,
                                        num_boost_rounds=n_rounds,
                                        early_stopping_rounds=early_stopping_rounds, verbose_eval=10
                                    )
        print(scores, best_num_rounds)
        print('Average best round: {}'.format(np.mean(best_num_rounds)))

    elif DEV == False:
        print('Mode: Submit')
        lgtrain = lgb.Dataset(X, y, feature_name=predictors, categorical_feature = categorical)
        del(X)
        gc.collect()

        lgb_clf = lgb.train(lgbm_params, lgtrain, num_boost_round=n_rounds, verbose_eval=40)
        lgpred = lgb_clf.predict(testing)

        lgsub = pd.DataFrame(lgpred,columns=["deal_probability"],index=testdex)
        lgsub['deal_probability'].clip(0.0, 1.0, inplace=True) # Between 0 and 1
        lgsub.to_csv(sub_filename,index=True,header=True)
    
    else:
        print('Mode: Dev without Cross-validation')
        ## DEV
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.10, random_state=23)
        lgtrain = lgb.Dataset(X_train, y_train, feature_name=predictors, categorical_feature = categorical)
        lgvalid = lgb.Dataset(X_valid, y_valid, feature_name=predictors, categorical_feature = categorical)
        del(X)
        del(X_train)
        gc.collect()

        lgb_clf = lgb.train(lgbm_params, lgtrain, num_boost_round=n_rounds, valid_sets=[lgtrain, lgvalid], valid_names=['train','valid'],
            early_stopping_rounds=early_stopping_rounds, verbose_eval=10)

        print('RMSE:', np.sqrt(metrics.mean_squared_error(y_valid, lgb_clf.predict(X_valid))))
        del(X_valid)
        gc.collect()

CPU times: user 0 ns, sys: 9 µs, total: 9 µs
Wall time: 16.5 µs


## Train Model: RNN for Text

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
num_cols = ['price', 'item_seq_number']
for col in num_cols:
    print('col: {}'.format(col))
    print(df[col].isnull().any())

col: price
False
col: item_seq_number
False


In [22]:
## TODO/ zeros in 'price' col ???

In [30]:
np.log1p(df['price'])

item_id
89a03cdb62c3    2.647760
b19c7b1221b4    1.925407
a9531307bfb5    2.139542
ea75ef7aac9d    2.666167
8e0368bccf5f    1.840283
1a9e736d9d5f    2.197932
46b9ab22e4d5    1.481884
e33ad17e3afd    1.793532
c31cbe9d818f    2.151867
5b219b622efe    2.201566
f3bfdc5d1659    1.555550
cb2aa2f2db4c    2.819080
5f1b7c04fa70    2.136220
bc4249121261    2.185329
3cc74acfd597    2.759381
e1fb8fcb76c4    2.280294
20a023fcf4e4    2.241968
9abf9d365ce7         NaN
a7552ff81fd8    2.526762
b598d199df17    1.723691
6ea51ad31c5c    2.067844
96fbefe55968    2.713228
26dee0228b3b    1.976108
47ed054ee87c    2.323401
af739158d7e4    2.021691
8dc057499a08    2.021691
e284ef01be04    1.875099
00d3943b935c    2.197932
f16256b3ed98    1.976108
d704c9b0af25    1.793532
                  ...   
bf8e9dd8e089         NaN
314d2145bf6b    1.591690
716d9c1eeb25    1.925407
efc230dc3cbe    2.001065
bf15a29a0e7f    2.702087
ba88539855df    1.840283
78db2e967e7f    2.039220
8eb6bc0a77fe    2.177481
65e4e9dbae0b    2

In [32]:
df.loc['9abf9d365ce7'].price

-999.0

In [38]:
%%time
if main_model == 'dnn':
    # inputs and preproc
    tokenizer_num_words = 200000
    cat_cols = ['region', 'city', 'category_name', 'parent_category_name',
                'param_1', 'param123', 'image_top_1']
    text_seq_cols = ['title_description']
    num_cols = ['price', 'item_seq_number'] #, 'price', 'item_seq_number']
    num_transform_spec = {
        'price': lambda s: np.log1p(np.clip(s, 0.0, float('+inf'))) # clip by almost 0, then convert to log
    } # dont transfor item_seq_number

    max_seq_length = 100
    text_spec = {}
    # embeddings
    embedding_file = '../data/cc.ru.300.vec'
    embedding_dim1 = 300 # dimension for the fasttext embeddings we've downloaded using the makefile.
    emb_out_size = 10 # embedding default output size for each embedding
    # model
    batch_size = 512*3
    model_type = RnnModelType.GRU # GRU or LSTM
    n_units = 50
    # final layers tuning
    dropout_0 = 0.1
    dropout_1 = 0.1
    ndense_0 = 512
    ndense_1 = 64
    final_layer_handler = None # dont override
    # control over learning and optimization, loss, metrics
    learning_rates = (0.009, 0.0045) # initial and final rates, evolution is exponential decay.
    loss_fn = None # rmse by default
    metrics_fns = None # [rmse] by default

    # Custom dataframe preparation for Keras
    def _prepare_df_handler(df):
        """ """
        df['param_1'].fillna(value='missing', inplace=True)
        df['param_1'] = df['param_1'].astype(str)
        df['param_2'].fillna(value='missing', inplace=True)
        df['param_2'] = df['param_2'].astype(str)
        df['param_3'].fillna(value='missing', inplace=True)
        df['param_3'] = df['param_3'].astype(str)
        df['title_description']= (df['title']+" "+df['description']).astype(str)
        df['param_1'].fillna(value='missing', inplace=True)
        df['param_2'].fillna(value='missing', inplace=True)
        df['param_3'].fillna(value='missing', inplace=True)
        df['param123'] = (df['param_1']+'_'+df['param_2']+'_'+df['param_3']).astype(str)
#        del(df['param_1']);  del(df['param_2']);  del(df['param_3']); gc.collect()
        return df
    
    # Model
    rnn = RnnTextModel(
        tokenizer_num_words,
        cat_cols=cat_cols,
        num_cols=num_cols, num_transform_spec=num_transform_spec,
        text_seq_cols=text_seq_cols, max_seq_length=max_seq_length,
        embedding_file=embedding_file, embedding_dim1=embedding_dim1, emb_out_size=emb_out_size,
        _prepare_df_handler=_prepare_df_handler,
        batch_size=batch_size, model_type=model_type, n_units=n_units,
        dropout_0=dropout_0, dropout_1=dropout_1, ndense_0=ndense_0, ndense_1=ndense_1,
        learning_rates=learning_rates, loss_fn=loss_fn, metrics_fns=metrics_fns, # using rmse by default
        final_layer_handler=final_layer_handler,
        optimizer="adam",
        text_spec={}
    )
    
    # Prepare dataframe for keras
    df = rnn.init_predictor(df, traindex)
    
    # Fit RNN + evaluate model
    v_rmse = rnn.fit(df.loc[traindex,:], y, n_iter=2, cv=False, test_size=0.10, random_state=23)

    # Make Predictions
    preds = rnn.predict(df.loc[testdex,:], verbose=1)

    # Save results
    sub = pd.DataFrame( columns = ['item_id', 'deal_probability'])
    sub['item_id'] = df.loc[testdex,:].index
    sub['deal_probability'] = preds
    sub.to_csv("rnn-output.csv", index=False)
    ## TODO: Add K-Fold

No loss, nor metrics specified: using rmse by default!
(236589, 300)
(236589, 300)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 169135 samples, validate on 18793 samples
Epoch 1/1
169135/169135 [==============================] - 321s 2ms/step - loss: 0.2538 - root_mean_squared_error: 0.2538 - val_loss: 0.2340 - val_root_mean_squared_error: 0.2340
Train on 169135 samples, validate on 18793 samples
Epoch 1/1
169135/169135 [==============================] - 299s 2ms/step - loss: 0.2336 - root_mean_squared_error: 0.2336 - val_loss: 0.2334 - val_root_mean_squared_error: 0.2334
0    False
dtype: bool
0    False
dtype: bool
rmse for validation set: 0.2333799017978882
63554/63554 [==============================] - 54s 849us/step
RNN Prediction is done.
(63554, 1)
CPU times: user 27min 48s, sys: 4mi

In [39]:
!head -n10 rnn-output.csv

item_id,deal_probability
00187a01adde,0.26224333
e49e7570f4c9,0.34400818
1fbd09de1edb,0.039779127
bcef66acf79f,0.12411514
5553436386df,0.29632738
65eaba16d707,0.05301839
ca105133c9f2,0.032180697
18a94481489a,0.44176075
8730b73938fc,0.14992341
